In [1]:
using JuMP
import HiGHS
import GLPK
using Clp

## Lista 1

### Questão 1

In [2]:
#model = Model(HiGHS.Optimizer)
model = Model(GLPK.Optimizer)

i = 2
j = 3
weights = [1, 1.5, 3.5, 2, 1, 2]

@variable(model, x[1:i, 1:j] >= 0)
@constraint(model, x[1,1] + x[2,1]   == 100)
@constraint(model, x[1,2] + x[2,2]  == 200)
@constraint(model, x[1,3] + x[2,3]  == 300)
@constraint(model, x[1,1] + x[1,2] + x[1,3] <= 400)
@constraint(model, x[2,1] + x[2,2] + x[2,3] <= 300)

#@objective(model, Min, sum(weights[n] * x[1:i, 1:j] for n in 1:6))
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:length(weights)))

x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]

In [3]:
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:i
    for n in 1:j
        println("x[$m, $n] = ", round(Int, value(x[m,n])))
    end
end

Valor Objetivo (Custo Mínimo): 1000.0
x[1, 1] = 100
x[1, 2] = 200
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 0
x[2, 3] = 300


Minimum Cost: 1000 

Generalizando para todos os casos:

$$ \begin{align} \min \; & c^T x \\
s.t \; & Ax = b \\
& Cx \leq d \\
& x \geq 0 
\end{align}$$

In [198]:
function LPP_Transporte(i, j, weights, A, C, b, d)
    model = Model(GLPK.Optimizer)
    p = length(weights)
    @variable(model, x[1:i, 1:j] >= 0)
    x = reshape(x', p, 1)
    for m in 1:length(b)
        @constraint(model, (A*x)[m] == b[m])
    end
    for n in 1:length(d)
        @constraint(model, (C*x)[n] <= d[n])
    end
    @objective(model, Min, sum(weights[k] * x[k] for k in 1:p))
    println("Função Objetivo: ", objective_function(model))
    optimize!(model)
    for k in 1:p
        #m,n = div(k-1, j) + 1, mod(k-1, j) + 1
        println(x[k]," = ", value(x[k]))
    end
    println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
    #print(x)
end

function print_var(x)
    if typeof(x) == Matrix{VariableRef}
        for i in 1:size(x,1)
            for j in 1:size(x,2)
                println(x[i,j], " = ", round(Int, value(x[i,j])))
            end
        end
    elseif typeof(x) == Vector{VariableRef}
        for i in 1:length(x)
            println(x[i]," = ", round(Int, value(x[i])))
        end
    elseif typeof(x) == VariableRef
        println(x, " = ", value(x))
    else 
        println("Tipo de variável não suportado")
    end
end

print_var (generic function with 1 method)

In [ ]:
LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: 50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]
x[1,1] = 20.0
x[1,2] = 0.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 10.0
x[2,3] = 30.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
Valor Objetivo (Custo Mínimo): 2500.0


Exemplificando com o problema anterior...

In [112]:
A = [1 0 0 1 0 0;
0 1 0 0 1 0;
0 0 1 0 0 1]
b = [100;200;300]

C = [1 1 1 0 0 0;
 0 0 0 1 1 1]
d = [400; 300]

weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3;

In [113]:
LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0
Valor Objetivo (Custo Mínimo): 1000.0


## Lista 2

### Problema da Mochila

In [193]:
n = 5;
capacity = 10.0;
profit = [5.0, 3.0, 2.0, 7.0, 4.0];
weight = [2.0, 8.0, 4.0, 2.0, 5.0];

#model = Model(GLPK.Optimizer)
model = Model(HiGHS.Optimizer)

@variable(model, x[1:n], Bin)

@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)

@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))

5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]

LPP:

In [194]:
print(model)

Max 5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]
Subject to
 2 x[1] + 8 x[2] + 4 x[3] + 2 x[4] + 5 x[5] <= 10
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary


In [195]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

Valor Objetivo (Custo Mínimo): 16.0


In [197]:
print("Itens Escolhidos: ", [i for i in 1:n if value(x[i]) == 1])

Itens Escolhidos: [1, 4, 5]

Maximum Profit = 16 \
Choosen Objects = 1,4,5

## Prova A1

### Questão 1 

In [186]:
model = Model(HiGHS.Optimizer)

i = 4
j = 3

# l_i: quantidade de líquido i revendido diretamente
@variable(model, l[1:i] >= 0)
# x_ij: quantidade de líquido i = 1,2,3,4 (A,B,C,D) usado na produção de j = 1,2,3 (E,F,G)
@variable(model, x[1:i, 1:j] >= 0)
# p: quantidade de líquido P produzido
@variable(model, p >= 0)

# make sum of x[m,j] <= 8000 as constraint
d = [8000, 4250, 16000, 2000]
for m in 1:i
    @constraint(model, l[m] + sum(x[m,j] for j in 1:j) <= d[m])
end

# Para produzir E, j=1
@constraint(model, x[1,1] == 0.3 * sum(x[m,1] for m in 1:i)) # A 
@constraint(model, x[2,1] >= 0.1 * sum(x[m,1] for m in 1:i)) # B
@constraint(model, x[3,1] == 0.4 * sum(x[m,1] for m in 1:i)) # C
@constraint(model, x[4,1] <= 0.05 * sum(x[m,1] for m in 1:i)) # D

# Para produzir F, j=2
@constraint(model, x[1,2] == 0.25 * sum(x[m,2] for m in 1:i)) # A
@constraint(model, x[2,2] <= 0.2 * sum(x[m,2] for m in 1:i)) # B
@constraint(model, x[3,2] == 0.2 * sum(x[m,2] for m in 1:i)) # C
@constraint(model, x[4,2] >= 0.1 * sum(x[m,2] for m in 1:i)) # D

# Para produzir G, j=3
@constraint(model, x[1,3] == 0.2 * sum(x[m,3] for m in 1:i)) # A
@constraint(model, x[2,3] >= 0.15 * sum(x[m,3] for m in 1:i)) # B
@constraint(model, x[3,3] == 0.4 * sum(x[m,3] for m in 1:i)) # C
@constraint(model, x[4,3] <= 0.2 * sum(x[m,3] for m in 1:i)) # D

# A produção de líquidos E,F,G deve suprir a obrigação da empresa
@constraint(model, sum(x[m,1] for m in 1:i) >= 400)
@constraint(model, sum(x[m,2] for m in 1:i) >= 800)
@constraint(model, sum(x[m,3] for m in 1:i) >= 200)

# O líquido P deve satisfazer as proporções especificadas de líquido E e G
@constraint(model, p/3 <= sum(x[m,1] for m in 1:i))
@constraint(model, 2*p/3 <= sum(x[m,3] for m in 1:i))

# Função Objetivo
@objective(model, Max, 6*(sum(l[m] for m in 1:i)) + 11*(sum(x[m,1] for m in 1:i) - p/3) + 15*(sum(x[m,2] for m in 1:i)) + 14*(sum(x[m,3] for m in 1:i) - 2*p/3)) + 22*p - 5.5*(l[1] + sum(x[1,n] for n in 1:j)) - 3.5*(l[2] + sum(x[2,j] for j in 1:j)) - 7.5*(l[3] + sum(x[3,j] for j in 1:j)) - 11.5*(l[4] + sum(x[4,j] for j in 1:j));

In [188]:
optimize!(model)
@assert is_solved_and_feasible(model)
print("Valor Objetivo (Lucro Máximo): ", objective_value(model))

Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [6e+00, 2e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 2e+04]
Solving LP without presolve, or with basis, or unconstrained
Model   status      : Optimal
Objective value     :  3.0450000000e+05
HiGHS run time      :          0.00
Valor Objetivo (Lucro Máximo): 304500.0

In [179]:
print_var(x)

x[1,1] = 120
x[1,2] = 200
x[1,3] = 2845
x[2,1] = 120
x[2,2] = 0
x[2,3] = 4130
x[3,1] = 160
x[3,2] = 160
x[3,3] = 5690
x[4,1] = 0
x[4,2] = 440
x[4,3] = 1560


In [180]:
print_var(p)

p = 0.0


In [181]:
print_var(l)

l[1] = 4835
l[2] = 0
l[3] = 9990
l[4] = 0


Lucro Máximo: 304500

### Questão 2

In [101]:
model = Model(HiGHS.Optimizer)

i = 3
j = 3
weights = [50, 90, 80, 0, 30, 40, 0, 60, 50]

@variable(model, x[1:i, 1:j] >= 0)

@constraint(model, x[1,1] == 20)
@constraint(model, x[1,2] + x[2,2] + x[3,2] == 10)
@constraint(model, x[1,3] + x[2,3] + x[3,3]  == 30)
@constraint(model, x[1,1] + x[1,2] + x[1,3]  <= 50)
@constraint(model, x[2,2] + x[2,3]  <= 50)
@constraint(model, x[3,2] + x[3,3]  <= 20)

#@objective(model, Min, sum(weights[n] * x[1:i, 1:j] for n in 1:6))
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:9))

50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]

In [102]:
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
print("Valor Objetivo (Custo Mínimo): ", objective_value(model))

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 9e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+01, 5e+01]
Presolving model
3 rows, 4 cols, 6 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-6); columns 0(-9); elements 0(-14) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.5000000000e+03
HiGHS run time      :          0.00
Valor Objetivo (Custo Mínimo): 2500.0

In [107]:
for m in 1:i
    for n in 1:j
        println("x[$m, $n] = ", round(Int, value(x[m,n])))
    end
end

x[1, 1] = 20
x[1, 2] = 0
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 10
x[2, 3] = 30
x[3, 1] = 0
x[3, 2] = 0
x[3, 3] = 0


Alternativamente...

In [104]:
A = [1 0 0 0 0 0 0 0 0;
0 1 0 0 1 0 0 1 0;
0 0 1 0 0 1 0 0 1]
b = [20; 10; 30]

C = [1 1 1 0 0 0 0 0 0 ;
0 0 0 0 1 1 0 0 0 ;
0 0 0 0 0 0 0 1 1]
d = [50; 50; 20];

In [105]:
LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: 50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]
x[1,1] = 20.0
x[1,2] = 0.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 10.0
x[2,3] = 30.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
Valor Objetivo (Custo Mínimo): 2500.0


Minimum Cost: 2500 \
Observations: Ilumination recieves only from eletricity, and space/water heating recieve from solar painels (its the cheapiest)